#### DialoGPT
DIALOGPT(Dialogue Generative Pre-trained Transformer)라는 크고 조정가능한 neural conversational response generation model을 소개합니다. 2005년부터 2017년 기간동안 Reddit comment chain에 추출한 1억4천7백만 대화형 코멘트에 학습된 DialoGPT는 automatic과(PPL같은) 싱글턴 대화 환경에서 human evaluation을 사람과 비슷한 성능을 얻는 것이 목표입니다.

DIALOGPT는 conversational neural response generation을 다루기 위해 GPT-2를 확장합니다. neural response generation은 자연스럽게 보이는 텍스트를 생성한다는 목표를 공유하는 text-generation의 하위분야입니다.

GPT-2와 같이 DIALOGPT는 autoregressive(AR) language model이며 모델 구성으로 multi-layer transformer를 사용합니다. 그러나 GPT-2와 다르게 Reddit discussion chain에서 추출된 대규모 대화 pair/sesion에서 학습됩니다. 논문에서 의도는 이 대규모 대화 pairs/session이 DIALOGPT가 대화 플로우에서 P(Target,Source)에대한 joint distribution를 캡쳐할 수 있게 하고자 한다는 겁니다.

##### Dataset
데이터셋은 2005년부터 2017년에 걸쳐 Reddit에서 스크랩된 comment chian에서 추출됩니다.
Reddit discussion은 스레드에 대한 응답 스레드는 하위 스레드의 루트 노드를 형성하기 때문에 자연스럽게 tree-structured한 응답 체인으로 펼쳐질 수 있습니다. 논문에서 root node에서 leaf node까지의 각각의 path(하위 스레드)를 대화의 멀티턴을 가진 학습 인스턴스로 사용합니다.

아래 기준에 해당하는 데이터를 필터링합니다.
(1) URL이 있는 source나 target

(2) 3개 이상의 단어 반복이 target에 포함된 경우

(3) 응답이 가장 자주 사용하는 top 50 단어에 적어도 하나 이상 포함하지 않은 경우(예를 들면 the, of, a)(이는 영어가 아닐 수도 있기 때문)

(4) 응답에 "[" 또는 "]"이 포함 된 경우 (이는 markup 언어 일 수도 있기 때문)

(5) source와 target 시퀀스가 합쳐서 200 단어보다 긴 경우

(6) target이 offensive language를 포함한 경우 (대용량 blcoklist에 매칭하는 방법으로 필터링)

(7) 하위 레딧에 많은 수가 offensive한 내용을 포함할 가능성이 많다고 인식되는 경우

(8) 단조로운 문장 적극적으로 배제 (1,000번 이상 본 tri-gram의 90%가 포함된 응답)

필터링 후 데이터 세트는 총 18억 개의 단어로 147,116,725개의 대화 인스턴스로 구성됩니다.

##### Mehthod
DialoGPT는 GPT-2와 같은 autoregressive LM으로, 차이점은 large-scale dialogue pairs/sessions extracted from Reddit discussion chains에서 훈련되었다는 점입니다. 
그리고 DialoGPT가 P(Target, Source)를 모델링할 수 있도록 Mutual Information Maximization를 사용합니다.

#### Mutual Information Maximization
저자는 source에 대해서만 조건화하여 응답을 생성하기 때문에 크게 상관 없는 일반적인 응답을 내놓는다고 이야기합니다. 그래서 역으로도 조건화하기 위해 다음을 최대화하도록 합니다.

<img src="./image/05_01.jpg" width="500" height="50">

여기서 λ는 얼마나 많이 generic responses에 대해 페널티를 줄 것인지를 제어하는 hyperparameter다.

In [ ]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from config import device_f, device_r, num_samples, MMI_temperature, top_k

torch.set_grad_enabled(False)

tokenizer = GPT2Tokenizer('medium/vocab.json', 'medium/merges.txt')

weights = torch.load('medium/medium_ft.pkl')
# fix misused key value
weights["lm_head.weight"] = weights["lm_head.decoder.weight"]
weights.pop("lm_head.decoder.weight", None)

cfg = GPT2Config.from_json_file('medium/config.json')
model: GPT2LMHeadModel = GPT2LMHeadModel(cfg)
model.load_state_dict(weights,strict=False)
if device_f == 'cuda':
    model.half()
model.to(device_f)
model.eval()

weights = torch.load('medium/small_reverse.pkl')
# fix misused key value
weights["lm_head.weight"] = weights["lm_head.decoder.weight"]
weights.pop("lm_head.decoder.weight", None)

reverse_model: GPT2LMHeadModel = GPT2LMHeadModel(cfg)
reverse_model.load_state_dict(weights,strict=False)
if device_r == 'cuda':
    reverse_model.half()
reverse_model.to(device_r)
reverse_model.eval()


end_token = torch.tensor([[50256]], dtype=torch.long)


def _get_response(output_token, past):
    out = torch.tensor([[]], dtype=torch.long, device=device_f)

    while True:
        util = model.forward(output_token, past_key_values=past)
        output_token, past = util['logits'],util['past_key_values']
        output_token = output_token[:, -1, :].float()
        indices_to_remove = output_token < torch.topk(output_token, top_k)[0][..., -1, None]
        output_token[indices_to_remove] = -float('Inf')
        output_token = torch.multinomial(F.softmax(output_token, dim=-1), num_samples=1)

        out = torch.cat((out, output_token), dim=1)

        if output_token.item() == end_token.item():
            break

    return out, past


def _score_response(output_token, correct_token):
    inputs = torch.cat((output_token, correct_token), dim=1)
    mask = torch.full_like(output_token, -100, dtype=torch.long)
    labels = torch.cat((mask, correct_token), dim=1)

    score = -reverse_model(inputs, labels=labels)['loss'].float()

    return score


def append_messages(old_list: list, new_list: list, truncate_length=64):
    for message in new_list:
        if message != '':
            input_token = tokenizer.encode(message, return_tensors='pt')
            input_token = torch.cat((input_token, end_token), dim=1)
            old_list.append(input_token)

    if len(old_list) == 0:
        old_list.append(end_token)

    # truncate
    total_length = 0
    for i, message in enumerate(reversed(old_list)):
        total_length += message.shape[1]
        if total_length > truncate_length:
            old_list[:] = old_list[-i:]


def generate_message(message_list: list, focus_last_message=True):
    total_input = torch.cat(message_list, dim=1).to(device_f)
    if focus_last_message:
        total_input_reversed = message_list[-1]
    else:
        total_input_reversed = torch.cat(list(reversed(message_list)), dim=1)

    past = None
    if total_input.shape[1] > 1:
        past = model(total_input[:, :-1])

    results = []
    for i in range(num_samples):
        result = _get_response(total_input[:, -1:], past['past_key_values'])
        score = _score_response(result[0].to(device_r), total_input_reversed.to(device_r))
        results.append(result + (score,))

    scores = torch.stack([x[2] for x in results], dim=0)
    winner = torch.multinomial(F.softmax(scores / MMI_temperature, dim=0), num_samples=1).item()
    # winner = torch.argmax(scores, dim=0)

    out = results[winner][0]

    return tokenizer.decode(out.tolist()[0], skip_special_tokens=True)


my_message_list = []
while True:
    print("usr >> ",end="")
    my_message = input()
    if my_message=="quit":
      print("bot >> Quit. Chating End")
      break
    append_messages(my_message_list, [my_message])
    my_response = generate_message(my_message_list)
    print('bot >>', my_response)

    append_messages(my_message_list, [my_response])